In [26]:
!wget http://opendata.deepsig.io/datasets/2016.10/RML2016.10b.tar.bz2

--2019-05-01 01:09:03--  http://opendata.deepsig.io/datasets/2016.10/RML2016.10b.tar.bz2
Resolving opendata.deepsig.io (opendata.deepsig.io)... 52.14.91.165
Connecting to opendata.deepsig.io (opendata.deepsig.io)|52.14.91.165|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1245608913 (1.2G) [application/x-bzip2]
Saving to: ‘RML2016.10b.tar.bz2.1’

RML2016.10b.tar.bz2 100%[===================>]   1.16G  8.91MB/s    in 1m 56s  

2019-05-01 01:10:59 (10.2 MB/s) - ‘RML2016.10b.tar.bz2.1’ saved [1245608913/1245608913]



In [0]:
import tarfile
tar = tarfile.open("RML2016.10b.tar.bz2", "r:bz2")
tar.extractall()
tar.close()

In [0]:
import numpy as np
import scipy.integrate as integrate
from sklearn.model_selection import train_test_split
import pickle
import keras.models as models
from keras.layers.convolutional import Convolution2D, ZeroPadding2D
from keras.layers.core import Dense, Reshape, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [0]:
# Load the dataset ...
#  You will need to seperately download or generate this file
Xd = pickle.load(open("RML2016.10b.dat",'rb'))
snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1,0])
raw_time_data = []  
data_labels = []
for mod in mods:
    for snr in snrs:
        raw_time_data.append(Xd[(mod,snr)])
        for i in range(Xd[(mod,snr)].shape[0]):  data_labels.append((mod,snr))
raw_time_data = np.vstack(raw_time_data)

In [0]:
derivative1_data = np.gradient(raw_time_data, axis=0)

In [0]:
integration_data = integrate.cumtrapz(raw_time_data, initial=0)

In [0]:
dataset = []
for i in range(len(raw_time_data)):
    dataset.append(np.vstack((raw_time_data[i], derivative1_data[i], integration_data[i])))

In [35]:
print("dataset size = %d" % len(dataset))
print("number of features for each instance = %d" % len(dataset[0]))
print("number of elements in each feature = %d" % len(dataset[0][0]))

dataset size = 1200000
number of features for each instance = 6
number of elements in each feature = 128


## Split the data into 50% for training/validation and 50% for testing.

In [38]:
training_data, test_data, training_labels, test_labels = train_test_split(dataset, data_labels, test_size = 0.5)

training_labels = [label[0] for label in training_labels]
test_labels = [label[0] for label in test_labels]

print("training data size: %d" % len(training_data))
print("test data size: %d" % len(test_data))
print("training data labels size: %d" % len(training_labels))
print("test data labels size: %d" % len(test_labels))

training data size: 600000
test data size: 600000
training data labels size: 600000
test data labels size: 600000


In [40]:
training_labels[1]

'AM-DSB'

In [0]:
label_encoder = LabelEncoder()
onehot_encoder = OneHotEncoder(sparse=False)


training_integer_encoded = label_encoder.fit_transform(np.array(training_labels))
training_integer_encoded = training_integer_encoded.reshape(len(training_integer_encoded), 1)
training_labels = onehot_encoder.fit_transform(training_integer_encoded)


test_integer_encoded = label_encoder.fit_transform(np.array(test_labels))
test_integer_encoded = training_integer_encoded.reshape(len(test_integer_encoded), 1)
test_labels = onehot_encoder.fit_transform(test_integer_encoded)


In [44]:
training_labels[5452]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

## Build the Convolutional Neural Network

In [45]:
cnn_model = models.Sequential([
    
    Reshape([1, 6, 128], input_shape = (6, 128)),
    
    Convolution2D(activation="relu", padding = 'same', data_format="channels_first", filters=64, kernel_size=(1, 3)),
    
    Convolution2D(activation="relu", padding = 'same', data_format="channels_first", filters=16, kernel_size=(2, 3)),
    
    Dropout(0.5),
    
    Flatten(),
    
    Dense(units=128, activation="relu"),  
    
    Dense(units=10, activation="softmax")
    
])

cnn_model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 1, 6, 128)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 6, 128)        256       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 6, 128)        6160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 6, 128)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 12288)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               1572992   
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1290      
Total para

In [0]:
cnn_model.fit(training_data, training_labels, epochs=100,
              batch_size=5120, validation_split= 0.01,
              callbacks=[
                  EarlyStopping(patience=3, monitor = 'val_loss', mode = 'min'),
                  ModelCheckpoint("cnn-best-model-{epoch:02d}-{val_acc:.2f}.hdf5", monitor='val_loss', save_best_only=True, mode='min', period=1)
              ])